In [10]:
import pandas as pd
import os

"""
PREPROCESS CSV

* -3 images sans masques 
* -BCP plus de fichier dans le csv que dans le dossiers/images/masques
* -FORMAT du nom différent dans le fichier du csv et dans le dossiers/images/masques (exemples: dans le csv -> ANG 5488,00 mais dans l'image -> ANG005488 (=trouver la jointure à faire)
* -convertir les floats en int, et Faire en sorte d'avoir 6 chiffres apres le trigramme(=ANG/AIX etc...), combler les chiffres manquants avec des 0.
* les feuilles acuminées on 3 types de valeur 1.0, 0.0, et -1.0 -> inutile d'avoir -1 et 0, donc convertir les -1 en 0

"""

base_dir = os.getcwd()
train_csv_file = os.path.join(base_dir, 'normalized.csv')
# Chemins des sous-dossiers pour train et test
train_image_dir = os.path.join(base_dir, 'image_herbier_final')

# Chemin du fichier CSV
 #train_csv_file = os.path.join(base_dir, 'Projet_DL.csv')

# Charger le fichier CSV
data = pd.read_csv(train_csv_file, sep=';')  # Assurez-vous d'utiliser le bon séparateur

def normalize_filename(name):
    # Vérifier si le nom contient une virgule, ce qui indique un nombre décimal
    if "," in name:
        # Retirer les espaces et remplacer la virgule par un point
        name = name.replace(" ", "").replace(",", ".")
        
        # Extraire la partie alphabétique et la partie numérique
        prefix = name[:3]  # "ANG"
        numeric_part = name[3:]  # Partie numérique

        # Convertir la partie numérique en entier pour enlever les décimales, puis ajouter des zéros initiaux
        num_part = str(int(float(numeric_part))).zfill(6)  # "004885" pour "4885" avec 6 chiffres
        return prefix + num_part
    else:
        # Si le nom est déjà un entier, le retourner sans modification
        return name


In [20]:
import os
import pandas as pd

# Chemins des fichiers et du dossier
base_dir = os.getcwd()
csv_file = os.path.join(base_dir, 'normalized.csv')  # Chemin vers le fichier CSV
image_dir = os.path.join(base_dir, 'image_herbier_final')  # Chemin vers le dossier contenant les images

# Charger le CSV
data = pd.read_csv(csv_file, sep=';')

# Vérifier que la colonne 'normalized_name' existe
if 'code' not in data.columns:
    raise ValueError("La colonne 'code' est absente du CSV.")

# Extraire les noms des fichiers dans le CSV (sans extension)
csv_files = set(data['normalized_name'])

# Lister les fichiers dans le dossier d'images (sans extension)
folder_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))}

# Identifier les fichiers présents dans les deux
intersection_files = csv_files & folder_files

# Identifier les fichiers supplémentaires dans le dossier
extra_files = folder_files - csv_files

# Identifier les fichiers manquants dans le dossier
missing_files = csv_files - folder_files

# Afficher les résultats
print(f"Nombre total de fichiers dans le CSV : {len(csv_files)}")
print(f"Nombre total de fichiers dans le dossier : {len(folder_files)}")
print(f"Nombre de fichiers présents dans les deux : {len(intersection_files)}")
print(f"Nombre de fichiers présents dans le dossier mais absents du CSV : {len(extra_files)}")
print(f"Nombre de fichiers présents dans le CSV mais absents du dossier : {len(missing_files)}")

# Facultatif : Enregistrer les résultats dans des fichiers
output_dir = os.path.join(base_dir, 'output_files')
os.makedirs(output_dir, exist_ok=True)

pd.DataFrame({'intersection_files': list(intersection_files)}).to_csv(os.path.join(output_dir, 'intersection_files.csv'), index=False)
pd.DataFrame({'extra_files': list(extra_files)}).to_csv(os.path.join(output_dir, 'extra_files_in_folder.csv'), index=False)
pd.DataFrame({'missing_files': list(missing_files)}).to_csv(os.path.join(output_dir, 'missing_files_in_folder.csv'), index=False)

print(f"Résultats enregistrés dans le dossier : {output_dir}")


Nombre total de fichiers dans le CSV : 2530
Nombre total de fichiers dans le dossier : 743
Nombre de fichiers présents dans les deux : 490
Nombre de fichiers présents dans le dossier mais absents du CSV : 253
Nombre de fichiers présents dans le CSV mais absents du dossier : 2040
Résultats enregistrés dans le dossier : c:\Users\akues\OneDrive\Documents\process\output_files


In [16]:
import os
import pandas as pd

# Définition des chemins
base_dir = os.getcwd()
csv_file = os.path.join(base_dir, 'Projet_DL.csv')
filtered_csv_file = os.path.join(base_dir, 'filtered_new_data.csv')

def normalize_filename(name):
    if not isinstance(name, str):
        return str(name)
    
    if "," in name:
        name = name.replace(" ", "").replace(",", ".")
        prefix = name[:3]
        numeric_part = name[3:]
        try:
            num_part = str(int(float(numeric_part))).zfill(6)
            return prefix + num_part
        except ValueError:
            return name
    else:
        return name

try:
    # Charger le fichier CSV
    data = pd.read_csv(csv_file, sep=';')
    
    # Vérifier si la colonne 'code' existe
    if 'code' not in data.columns:
        raise ValueError("La colonne 'code' n'existe pas dans le CSV")
    
    # Appliquer la normalisation uniquement à la colonne 'code'
    data['normalized_name'] = data['code'].apply(normalize_filename)
    
    # Afficher les deux colonnes côte à côte
    comparison_df = data[['code', 'normalized_name']].head(20)
    print("\nComparaison des 20 premiers codes originaux et normalisés :")
    print("-" * 50)
    print(comparison_df)
    
    # Afficher quelques statistiques
    print("\nStatistiques :")
    print("-" * 50)
    print(f"Nombre total de lignes : {len(data)}")
    print(f"Nombre de codes uniques avant normalisation : {data['code'].nunique()}")
    print(f"Nombre de codes uniques après normalisation : {data['normalized_name'].nunique()}")
    
    # Afficher les cas où la normalisation a changé le code
    changed = data[data['code'] != data['normalized_name']]
    print("\nExemples de codes qui ont été modifiés :")
    print("-" * 50)
    print(changed[['code', 'normalized_name']].head())
    
    # Enregistrer le CSV avec la nouvelle colonne normalisée
    filtered_csv_file = os.path.join(base_dir, 'data_with_normalized_codes.csv')
    data.to_csv(filtered_csv_file, index=False, sep=';')
    print(f"\nCSV avec codes normalisés enregistré sous : {filtered_csv_file}")

except FileNotFoundError:
    print(f"Erreur : Le fichier CSV '{csv_file}' n'a pas été trouvé")
except ValueError as e:
    print(f"Erreur : {e}")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")


Comparaison des 20 premiers codes originaux et normalisés :
--------------------------------------------------
               code normalized_name
0   CLF 289236,0000       CLF289236
1         P03327766       P03327766
2         P04681621       P04681621
3         P03550384       P03550384
4         P01902590       P01902590
5         P02520870       P02520870
6         P03010502       P03010502
7         LY0272200       LY0272200
8         P05259017       P05259017
9         P02938082       P02938082
10        NCY014503       NCY014503
11        LY0237730       LY0237730
12     ANG 15579,00       ANG015579
13        P04146284       P04146284
14        P00412460       P00412460
15        P05238388       P05238388
16  CLF 289227,0000       CLF289227
17        P05156616       P05156616
18       MPU1166167      MPU1166167
19        VTA017420       VTA017420

Statistiques :
--------------------------------------------------
Nombre total de lignes : 2530
Nombre de codes uniques avant norma